In [39]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer

from IPython.display import display

# Wprowadzenie

Celem tego notebooka jest wprowadzenie do tematyki porównywania ze sobą klaysfikatorów. Nie jest to sprawa tak prosta, jak mogłoby się wydawać. 

Ważne źródła informacji:

> Dietterich, T. G. (1998) „Approximate statistical tests for comparing supervised classification learning  algorithms”, Neural computation. MIT Press, 10(7), ss. 1895–1923

> Demšar, J. (2006) „Statistical comparisons of classifiers over multiple data sets”, Journal of Machine learning research, 7(Jan), ss. 1–30.


Il. algorytmów \ Il. zbiorów danych | 1 | 1+ |
------------- | ------------- | ------------- |
2 | T-test różnicy metryki CV <br/> <br/> LUB <br/> <br/> Test McNemara | Test różnicy rang Wilcoxon'a |
2+ | T-test różnicy metryki <br/> <br/> LUB <br/><br/> Test Friedmana | Test Friedmana |

W czym jest problem? Dlaczego nie można zawsze stosować zwykłego t-testu różnicy metryki?

* Nie są spełnione 

# Przygotowanie metryki

Metryka F1 dostarczana przez bibliotekę może działać tylko z dwiema klasami. Żeby używać jej dla więcej niż 2 klas trzeba to zrobić po swojemu :) 

In [26]:
score_f = lambda y_true, y_predicted: f1_score(y_true=y_true, y_pred=y_predicted, average='weighted')
f1_scorer = make_scorer(score_f)

# Sytuacja: 2 klasyfikatory, 1 zbiór danych (2x1)

In [27]:
data1 = pd.read_csv("./iris.csv")
data1.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [28]:
# Podział na zmienne egzogeniczne i endogeniczne

X, Y = data1.drop("species", axis=1), data1.species

In [29]:
display(X.head(3))

display(Y.head(3))

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


0    setosa
1    setosa
2    setosa
Name: species, dtype: object

In [66]:
# Utwórz obiekt klasyfikatora drzewa decyzyjnego i klasyfikatora Knn. Parametry dowolne

# niektóre klasyfikatory wymagają dodatkowego zapewnienia 
# deterministyczności. Trzeba to zawsze sprawdzać w dokumentacji
dt = DecisionTreeClassifier(random_state=555)
knn = KNeighborsClassifier()

## Zbieranie metryk

 Użyjemy 10-o krotnej walidacji krzyżowej

Kroki:

* Przygotowujemy *środowisko eksperymentalne* - podział danych ma być **deterministyczny** tzn. za każdym razem dawać taki sam wynik
* Używamy próbkowania **proporcjonalnego** tzn. zachowującego proporcje klas z oryginalnych danych

In [84]:
# Używamy obiektu, który zapewni nam deterministyczny podział danych
cv = StratifiedKFold(
    n_splits=10,        # Ustawiamy ilość losowań
    random_state=555    # Ustawiamy "ziarno" - dowolna liczba, ważne żeby pamiętać, czego się użyło :D 
)

### Wersja uproszczona

Ta wersja jest szybsza, także przydaje się podczas prototypowania wstępnego. Niestety, ma poważne ograniczenia:
1. nie pozwala na dokładne kontrolowanie, w jaki sposób działają klasyfikatory
2. nie pozwala liczyć dodatkowych metryk w trakcie działania
3. nie mamy wpływu na obsługę danych

In [79]:
# Używamy wbudowanej funkcji do policzenia wyników dla drzewa
dt_score_auto = cross_val_score(
    dt,                     # nasz klasyfikator
    X,                      # macierz zmiennych egzogenicznych
    Y,                      # wektor etykiet/klasy docelowej
    scoring=f1_scorer,      # nasza funkcja licząca metrykę F1
    cv=cv,                  # 10-o krotna walidacja
)

In [80]:
dt_score_auto

array([1.        , 0.93265993, 1.        , 0.93265993, 0.93265993,
       0.86666667, 0.93265993, 0.93265993, 1.        , 1.        ])

In [82]:
# Używamy wbudowanej funkcji do policzenia wyników dla Knn
knn_score_auto = cross_val_score(
    knn,                    # nasz klasyfikator
    X,                      # macierz zmiennych egzogenicznych
    Y,                      # wektor etykiet/klasy docelowej
    scoring=f1_scorer,      # nasza funkcja licząca metrykę F1
    cv=cv,                  # 10-o krotna walidacja
)

In [83]:
knn_score_auto

array([1.        , 0.93265993, 1.        , 1.        , 0.86111111,
       0.93265993, 0.93265993, 1.        , 1.        , 1.        ])

### Wersja zaawansowana

Wersja rekomendowana :) Pozwala nam na dokładne kontrolowanie liczonych metryk i przygotowywanie pewnych rzeczy "na boku". 

Kroki:

1. przygotowujemy deterministyczny podział danych - to mieliśmy zrobione już wyżej :)
2. przygotowujemy listę klasyfikatorów - jeśli mamy tylko dwa to sprawa jest łatwiejsza :) 
3. dzielimy dane k-razy
4. dla każdej iteracji i dla każdego klasyfikatora, liczymy jego trafność, zapisujemy wyniki i zgodność z wartościami oczekiwanymi

In [99]:
expected = []

dt_predictions_manual = []
dt_scores_manual = []

knn_predictions_manual = []
knn_scores_manual = []

In [100]:
# Krok 3 i 4:

# Manualnie dzielimy dane wg. deterministycznego próbkowania
for train_idx, test_idx in cv.split(X, Y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    # szkolimy klasfikatory
    # drzewo decyzyjne
    dt_fold = DecisionTreeClassifier(random_state=555)
    dt_fold.fit(X_train, y_train)
    
    # Knn
    knn_fold = KNeighborsClassifier()
    knn_fold.fit(X_train, y_train)
    
    # dokonujemy predykcji na zbiorze testowym
    dt_fold_predict = dt_fold.predict(X_test)
    knn_fold_predict = knn_fold.predict(X_test)
    
    # zapisujemy predykcje - to się nam bardzo przyda
    expected.append(y_test)
    dt_results_manual.append(dt_fold_predict)
    knn_results_manual.append(knn_fold_predict)
    
    # liczymy i zapisujemy trafności - to też się zaraz przyda
    dt_fold_score = f1_score(y_true=y_test, y_pred=dt_fold_predict, average='weighted')
    knn_fold_score = f1_score(y_true=y_test, y_pred=knn_fold_predict, average='weighted')
    
    dt_scores_manual.append(dt_fold_score)
    knn_scores_manual.append(knn_fold_score)
    

## Analiza metryk - 2x1

### Metoda 1 - t-test różnicy średnich metryk